<div style="background-color:#000;"><img src="pqn.png"></img></div>

### Import necessary libraries

We import the required libraries for our pairs trading strategy implementation.

In [ ]:
import os
import asyncio
from dotenv import load_dotenv

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import (
    FunctionAgent,
    AgentWorkflow,
    AgentOutput,
    ToolCall,
    ToolCallResult,
)
from llama_index.core import SimpleDirectoryReader, GPTVectorStoreIndex

These libraries provide essential tools for our project. We use them for environment variable management, asynchronous programming, and leveraging AI models for natural language processing tasks. The llama_index library offers specialized components for building AI-powered workflows and indexing documents.

### Set up the environment and initialize the language model

In [ ]:
load_dotenv()

In [ ]:
llm = OpenAI(model="gpt-4o")

We load environment variables and initialize our language model. This setup ensures we have access to necessary credentials and creates an instance of the OpenAI language model we'll use throughout our workflow.

### Define tool functions for our workflow

In [ ]:
async def read_pdf_tool(ctx: Context) -> str:
    documents = SimpleDirectoryReader(input_files=["pairs.pdf"]).load_data()
    index = GPTVectorStoreIndex(documents)
    query_engine = index.as_query_engine()
    query = (
        "Extract a detailed description of the pairs trading strategy implementation from this PDF. "
        "Ensure the description is detailed enough to reproduce the strategy in code."
    )
    response = query_engine.query(query)
    return str(response)

In [ ]:
async def build_plan_tool(ctx: Context, plan: str) -> str:
    current_state = await ctx.get("state")
    current_state["implementation_plan"] = plan
    await ctx.set("state", current_state)
    return "Implementation plan recorded."

In [ ]:
async def write_code_tool(ctx: Context, plan: str) -> str:
    current_state = await ctx.get("state")
    current_state["python_code"] = plan
    await ctx.set("state", current_state)
    return "Python code recorded."

We define three asynchronous functions that serve as tools in our workflow. The read_pdf_tool extracts information from a PDF file, the build_plan_tool creates an implementation plan, and the write_code_tool generates Python code based on the plan. These tools interact with the workflow context to update the state as they progress.

### Create function agents for our workflow

In [ ]:
pdf_reader_agent = FunctionAgent(
    name="PDFReaderAgent",
    description=(
        "Reads a PDF file containing a pairs trading strategy and extracts a detailed description "
        "of the strategy implementation."
    ),
    system_prompt=(
        "You are the PDFReaderAgent that can read PDFs containing implementation details of pairs trading strategies "
        "and describe the strategy in detail. Once you read the PDF and describe the implementation details and are "
        "satisfied, you should hand off control to the PlanBuilderAgent to develop an implementation plan. "
        "You should have a detailed description of the strategy before handing off control to the PlanBuilderAgent."
    ),
    llm=llm,
    tools=[read_pdf_tool],
    can_handoff_to=["PlanBuilderAgent"],
)

In [ ]:
plan_builder_agent = FunctionAgent(
    name="PlanBuilderAgent",
    description=(
        "Takes the detailed strategy description and builds a detailed plan to implement the strategy in Python."
    ),
    system_prompt=(
        "You are the PlanBuilderAgent. Your task is to analyze the strategy description from the state and generate "
        "a detailed plan outlining the steps, functions, and code structure required to implement the pairs trading strategy. "
        "Include suggested Python libraries for the implementation. Your plan should be in markdown format. Once the plan "
        "is written, you should hand off control to the CodeWriterAgent. Output your plan with no preamble. Just output the plan."
    ),
    llm=llm,
    tools=[build_plan_tool],
    can_handoff_to=["CodeWriterAgent"],
)

In [ ]:
code_writer_agent = FunctionAgent(
    name="CodeWriter

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.